# Step1: 추천 후보군 생성
1. 라우터로 유저 고유 아이디 들어옴
2. 이 고유 아이디 들어오면 장우님 뉴스 로그 api로 
{
      "timestamp": "2025-06-18T17:22:03",
      "timestampRaw": "2025-06-18 17:22:03",
      "thread": "http-nio-8080-exec-3",
      "level": "INFO",
      "logger": "c.n.n.a.n.v2.impl.ml.GetDetailNewsV2",
      "message": "2025-06-18 17:22:03 [http-nio-8080-exec-3] [INFO ] c.n.n.a.n.v2.impl.ml.GetDetailNewsV2 - [memberId : anonymous]  [newsId : 20250618_28735495]",
      "memberId": "anonymous",
      "newsId": "20250618_28735495" -> 이게 클릭 뉴스 아이디 됨
    },
이거 받아옴
뉴스 상세 메타데이터 조회로 데이터 미리 불러와서 클릭 뉴스 df 만들어 놓기 
{
  "news_id": "string",
  "summary": "string",
  "stock_list": [],
  "stock_list_view": [],
  "industry_list": [],
  "impact_score": 0
}

2-1. 뉴스 로그 api 없는 경우도 고려
4. 주요 뉴스 목록 조회로 3개월치 받아오기 -> 얘가 후보 뉴스 아이디
5. onnx 모델 불러와서 돌려 two-tower -> 클릭 뉴스 임베딩(n), 후보 뉴스 임베딩(n) 리턴은 후보뉴스 여러개의 재정렬된 리스트
6. 10개 걸러서 news_id 빼기

# Step2: 추천 후보군 리랭킹
1. 유저 아이디 이용해서 유저 정보 조회 api로 userPnl, asset, investScore 가져오기
2. 후보군 뉴스 상세 메타데이터 조회로 스톡리스트, 인더스트리 리스트, 임팩트 스코어, 토픽 db에서 끌고와서 붙이고 가장 큰값으로 메인토픽도 붙이기
4. 나온 결과로 내 모델에 넣어서 리랭킹 4개


# Step1

----------------------------------------------

1. 유저 고유아이디 수집

2. 뉴스 로그 api

In [95]:
import pandas as pd

click_log_df = pd.read_csv("C:/Users/user/fin_project/eda/hyejin/click_log.csv")

# user_id가 'u1'인 행만 남기기
click_log_df = click_log_df[click_log_df['user_id'] == 'u1']

# 그리고 clicked가 1인 news_id 리스트 추출
u1_clicked_news = click_log_df.loc[click_log_df['clicked'] == 1, 'news_id'].tolist()

print(u1_clicked_news)

['20250523_0028', '20250523_0067', '20250523_0075', '20250523_0087', '20250523_0109', '20250523_0111', '20250522_0025', '20250522_0034', '20250522_0050', '20250522_0057', '20250522_0072', '20250522_0075', '20250522_0089', '20250522_0099', '20250522_0104', '20250522_0109', '20250522_0116', '20250522_0120', '20250522_0155', '20250522_0173', '20250521_0056', '20250521_0105', '20250521_0117', '20250521_0130', '20250521_0165', '20250521_0171', '20250521_0173', '20250520_0032', '20250520_0171', '20250520_0174', '20250519_0102', '20250519_0149', '20250519_0164', '20250519_0171', '20250516_0065', '20250516_0069', '20250516_0117', '20250516_0130', '20250516_0134', '20250515_0010', '20250515_0137', '20250515_0174', '20250515_0190', '20250514_0015', '20250514_0020', '20250514_0051', '20250514_0062', '20250514_0143', '20250514_0147', '20250514_0155', '20250513_0017', '20250513_0018', '20250513_0052', '20250513_0068', '20250513_0084', '20250512_0016', '20250512_0032', '20250509_0032', '20250509_005

In [21]:
u1_clicked_news = u1_clicked_news[:30]

In [22]:
u1_clicked_news

['20250523_0028',
 '20250523_0067',
 '20250523_0075',
 '20250523_0087',
 '20250523_0109',
 '20250523_0111',
 '20250522_0025',
 '20250522_0034',
 '20250522_0050',
 '20250522_0057',
 '20250522_0072',
 '20250522_0075',
 '20250522_0089',
 '20250522_0099',
 '20250522_0104',
 '20250522_0109',
 '20250522_0116',
 '20250522_0120',
 '20250522_0155',
 '20250522_0173',
 '20250521_0056',
 '20250521_0105',
 '20250521_0117',
 '20250521_0130',
 '20250521_0165',
 '20250521_0171',
 '20250521_0173',
 '20250520_0032',
 '20250520_0171',
 '20250520_0174']

In [ ]:
# def extract_news_ids(api_response: dict) -> list:
#     """
#     뉴스 로그 API 응답에서 newsId 추출
#     :param api_response: API 응답 딕셔너리
#     :return: newsId 목록 (중복 포함)
#     """
#     # 데이터 존재 여부 확인
#     if not api_response.get("Success", False):
#         return []
    
#     # data 필드 추출
#     log_data = api_response.get("data", [])
    
#     # newsId 추출
#     news_ids = []
#     for log_entry in log_data:
#         if "newsId" in log_entry:
#             news_ids.append(log_entry["newsId"])
    
#     return news_ids

# ##################################################

# 이거 코드 분리해야겠는데?
newsId 받아와서 data 필드 추출하고 df로 만들어서 newsid만 추출하는


# news_id_list = extract_news_ids(api_response)
# print(news_id_list)  # 출력: ['20250513_0092', '20250618_28735495']


['20250513_0092', '20241015_0007', '20240529_0117', '20240725_0183', '20250513_0092', '20250513_0092', '20240108_0278', '20250513_0085', '20250513_0094', '20241021_0109', '20250513_0094', '20250513_0094', '20250513_0094', '20250513_0094', '20250513_0094', '20241204_0007', '20250513_0092', '20250513_0092', '20250513_0092', '20250514_0151', '20250513_0094', '20250513_0094', '20250523_0004', '20250523_0004', '20250523_0004', '20250523_0004', '20240715_0269', '20250523_0004', '20250523_0004', '20250523_0004', '20250523_0004', '20250523_0004', '20250523_0004', '20250523_0004', '20250523_0004', '20250523_0004', '20250514_0151', '20240521_0179', '20250523_0007', '20250523_0007', '20250523_0007', '20250523_0007', '20250523_0007', '20250523_0007', '20250523_0007', '20250523_0007', '20250523_0007', '20250523_0002', '20250523_0004', '20250523_0002', '20250523_0002', '20250523_0002', '20250523_0002', '20250523_0015', '20250523_0015', '20250523_0032', '20250523_0032', '20250523_0010', '20250523_001

In [ ]:
# import requests
# from datetime import datetime

# # API 설정
# API_BASE_URL = "http://43.200.17.139:8080"
# NEWS_LOGS_ENDPOINT = "/api/newsLogs"

# def get_news_logs(member_id: str, start_date: str, end_date: str) -> dict:
#     """
#     뉴스 로그 API 호출
#     :param member_id: 멤버 ID
#     :param start_date: 시작 날짜 (YYYY-MM-DD 형식)
#     :param end_date: 종료 날짜 (YYYY-MM-DD 형식)
#     :return: API 응답 딕셔너리
#     """
#     url = API_BASE_URL + NEWS_LOGS_ENDPOINT
#     params = {
#         "startDate": start_date,
#         "endDate": end_date,
#         "memberId": member_id
#     }
    
#     try:
#         response = requests.get(url, params=params)
#         response.raise_for_status()
#         return response.json()
#     except Exception as e:
#         print(f"API 호출 실패: {str(e)}")
#         return {"Success": False, "data": []}

# def extract_news_ids(api_response: dict) -> list:
#     """
#     뉴스 로그 API 응답에서 newsId 추출
#     :param api_response: API 응답 딕셔너리
#     :return: newsId 목록 (중복 포함)
#     """
#     if not api_response.get("Success", False):
#         return []
    
#     log_data = api_response.get("data", [])
#     news_ids = []
#     for log_entry in log_data:
#         if "newsId" in log_entry:
#             news_ids.append(log_entry["newsId"])
    
#     return news_ids

# def collect_member_news_ids(member_id: str, start_date: str, end_date: str) -> list:
#     """
#     특정 멤버의 뉴스 ID 목록 수집
#     :param member_id: 멤버 ID (예: "anonymous", "user123")
#     :param start_date: 조회 시작 날짜
#     :param end_date: 조회 종료 날짜
#     :return: newsId 목록
#     """
#     # API 호출
#     api_response = get_news_logs(member_id, start_date, end_date)
    
#     # newsId 추출
#     news_ids = extract_news_ids(api_response)
    
#     print(f"멤버 '{member_id}': {len(news_ids)}개의 뉴스 로그 발견")
#     return news_ids

In [ ]:
# # 파라미터 설정
# member_id = "anonymous"  # 또는 실제 멤버 ID
# start_date = "2025-06-01"
# end_date = "2025-06-19"
    
# # 뉴스 ID 수집
# news_ids = collect_member_news_ids(member_id, start_date, end_date)
    
# print(f"수집된 뉴스 ID: {news_ids}")
# print(f"총 개수: {len(news_ids)}")
    
# # 중복 제거한 고유 뉴스 ID
# unique_news_ids = list(set(news_ids))
# print(f"고유 뉴스 ID 개수: {len(unique_news_ids)}")


멤버 'anonymous': 163개의 뉴스 로그 발견
수집된 뉴스 ID: ['20250513_0092', '20250618_28735495', '20241015_0007', '20240529_0117', '20240725_0183', '20250618_07945335', '20250513_0092', '20250513_0092', '20250618_44957917', '20250618_07945335', '20240108_0278', '20250618_44957917', '20250618_07945335', '20250618_74048781', '20250513_0085', '20250513_0094', '20250618_24658192', '20241021_0109', '20250513_0094', '20250513_0094', '20250513_0094', '20250513_0094', '20250618_47916909', '20250513_0094', '20241204_0007', '20250513_0092', '20250618_70753735', '20250513_0092', '20250513_0092', '20250618_70753735', '20250514_0151', '20250513_0094', '20250513_0094', '20250523_0004', '20250523_0004', '20250523_0004', '20250523_0004', '20240715_0269', '20250523_0004', '20250523_0004', '20250523_0004', '20250523_0004', '20250523_0004', '20250523_0004', '20250523_0004', '20250523_0004', '20250523_0004', '20250514_0151', '20240521_0179', '20250523_0007', '20250523_0007', '20250523_0007', '20250523_0007', '20250523_00

이 뉴스 아이디 리스트 이용해서 본문 뽑아야됨

In [24]:
import requests
import pandas as pd

API_BASE_URL = "http://3.37.207.16:8000"
METADATA_ENDPOINT = "/news/v2/{news_id}/metadata"

def get_news_metadata(news_id):
    url = API_BASE_URL + METADATA_ENDPOINT.format(news_id=news_id)
    response = requests.get(url)
    response.raise_for_status()
    return response.json()

def get_metadata_df(news_ids):
    records = []
    for news_id in news_ids:
        try:
            meta = get_news_metadata(news_id)
            records.append(meta)
        except Exception as e:
            print(f"{news_id} 처리 중 오류: {e}")
    return pd.DataFrame(records)


# DataFrame 생성 예시
clicked_df = get_metadata_df(u1_clicked_news)

clicked_df.head()


,news_id,summary,stock_list,stock_list_view,industry_list,impact_score
0,20250523_0028,삼성바이오로직스는 전날 22일 바이오의약품 개발 및 상업화 사업을 분할해 신설법인(...,"[{'stock_id': '207940', 'stock_name': '삼성바이오로직...","[{'stock_id': '207940', 'stock_name': '삼성바이오로직...","[{'stock_id': '207940', 'industry_id': '32101'...",0.09
1,20250523_0067,삼성바이오로직스는 단순·인적분할 방식으로 삼성바이오에피스홀딩스(가칭)를 설립해 바이...,"[{'stock_id': '207940', 'stock_name': '삼성바이오로직...","[{'stock_id': '207940', 'stock_name': '삼성바이오로직...","[{'stock_id': '207940', 'industry_id': '32101'...",0.09
2,20250523_0075,DS투자증권은 인적분할한 삼성바이오로직스(이하 삼바) 주식에 대해 적정 기업가치를 ...,"[{'stock_id': '207940', 'stock_name': '삼성바이오로직...","[{'stock_id': '207940', 'stock_name': '삼성바이오로직...","[{'stock_id': '207940', 'industry_id': '32101'...",0.09
3,20250523_0087,"23일 NH투자증권은 올해 하반기 제약·바이오 산업의 전망과 관련해 ""미국 헬스케어...","[{'stock_id': '005940', 'stock_name': 'NH투자증권'...","[{'stock_id': '005940', 'stock_name': 'NH투자증권'...","[{'stock_id': '005940', 'industry_id': '116601...",0.09
4,20250523_0109,키움증권은 23일 삼성바이오로직스가 의약품 위탁 개발·생산(CDMO) 부문은 존속법...,"[{'stock_id': '039490', 'stock_name': '키움증권'},...","[{'stock_id': '039490', 'stock_name': '키움증권'},...","[{'stock_id': '039490', 'industry_id': '116601...",0.09


In [33]:
clicked_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   news_id          30 non-null     object 
 1   summary          30 non-null     object 
 2   stock_list       30 non-null     object 
 3   stock_list_view  30 non-null     object 
 4   industry_list    30 non-null     object 
 5   impact_score     30 non-null     float64
 6   embedding        30 non-null     object 
dtypes: float64(1), object(6)
memory usage: 1.8+ KB


4. 주요 뉴스 목록 조회로 3개월치 받아오기 -> 얘가 후보 뉴스 아이디

In [28]:
import requests
import pandas as pd

# API 기본 설정
API_BASE_URL = "http://3.37.207.16:8000"
HIGHLIGHTDATA_ENDPOINT = "/news/v2/highlights"

# 파라미터 설정
params = {
    "start_datetime": "2025-02-15T00:00:00",
    "end_datetime": "2025-05-16T00:00:00",
    "limit": 50,
    "stock_list": ""  # 종목코드가 필요하면 여기에 입력 (예: "005930,000660")
}

# GET 요청
response = requests.get(API_BASE_URL + HIGHLIGHTDATA_ENDPOINT, params=params)
response.raise_for_status()  # 오류 발생 시 예외

# JSON 데이터 추출
data = response.json()

# DataFrame 변환
candi_df = pd.DataFrame(data)

candi_df.head(50)


,news_id,wdate,title,image,press,summary,impact_score,url
0,20250429_0099,2025-04-29T10:05:00,"한덕수 대선 출마 임박설에 관련주 급등…일정실업, 이틀 연속 上",https://imgnews.pstatic.net/image/009/2025/04/...,매일경제,한덕수 대통령 권한대행 국무총리의 대선 출마가 임박한 것으로 알려지며 일정실업은 고...,1.44,https://n.news.naver.com/mnews/article/009/000...
1,20250414_0083,2025-04-14T09:42:00,오세훈 대선 불출마에 테마주 '와르르',https://imgnews.pstatic.net/image/215/2025/04/...,한국경제TV,윤석열 전 대통령 탄핵 이후 치러지는 조기 대선 유력 후보로 꼽혀 오세훈 서울시장이...,0.86,https://n.news.naver.com/mnews/article/215/000...
2,20250513_0089,2025-05-13T09:31:00,"한진칼, '쩜상'…추격매수냐 vs 익절이냐 '분수령'",https://imgnews.pstatic.net/image/215/2025/05/...,한국경제TV,호반건설이 지난 3월 한진칼 주주총회에 참석해 이사 보수 한도를 90억원에서 120...,0.78,https://n.news.naver.com/mnews/article/215/000...
3,20250513_0148,2025-05-13T00:25:00,"호반건설, 한진칼 지분 더 샀다…조원태와 1.5%P差",https://ssl.pstatic.net/static.news/image/news...,한국경제,호반건설이 12일 한진칼 보유 지분을 17.44%에서 18.46%로 1.02%포인트...,0.78,https://n.news.naver.com/mnews/article/015/000...
4,20250513_0094,2025-05-13T09:16:00,"호반건설 지분 매입에, 한진칼 장 초반 약 30% 강세 [특징주]",https://imgnews.pstatic.net/image/014/2025/05/...,파이낸셜뉴스,호반건설이 한진그룹 지주사 한진칸 지분율을 18% 이상으로 높였다는 소식에 한진칼이...,0.78,https://n.news.naver.com/mnews/article/014/000...
5,20250513_0092,2025-05-13T09:17:00,호반건설 한진그룹 경영 개입할까?…한진칼 상한가행,https://imgnews.pstatic.net/image/008/2025/05/...,머니투데이,호반건설은 한진칼 보유지분을 1%p(포인트)가량 추가로 사들여 17.44%에서 18...,0.78,https://n.news.naver.com/mnews/article/008/000...
6,20250513_0085,2025-05-13T09:53:00,"한진칼, 호반건설 지분 추가매입 소식에 ‘上’…경영권 분쟁 가능성",https://imgnews.pstatic.net/image/009/2025/05/...,매일경제,호반건설이 대한항공을 거느린 한진그룹 지주사인 한진칼 지분을 추가로 매입했다는 소식...,0.78,https://n.news.naver.com/mnews/article/009/000...
7,20250513_0041,2025-05-13T15:00:00,국내 유일 대형항공사 대한한공 경영권 분쟁 가능성에 ··· 대한항공 모회사 한진칼 ...,https://imgnews.pstatic.net/image/009/2025/05/...,매일경제,호반건설이 한진칼 지분을 추가로 매입해 지분율(특수관계인 포함)이 17.44%에서 ...,0.78,https://n.news.naver.com/mnews/article/009/000...
8,20250417_0067,2025-04-17T10:01:00,"제이준코스메틱, 프리미엄 'PDRN 포슬린 앰플' 신제품 출시",https://imgnews.pstatic.net/image/008/2025/04/...,머니투데이,제이준코스메틱은 연어나 송어의 생식세포에서 DNA 조각을 추출한 물질인 PDRN(폴...,0.75,https://n.news.naver.com/mnews/article/008/000...
9,20250430_0098,2025-04-30T09:38:00,상조 1위 회사 인수 소식에…웅진 15%대 급등,https://imgnews.pstatic.net/image/008/2025/04/...,머니투데이,웅진이 국내 1위 상조 업체인 프리드라이프를 인수한다는 소식에 장 초반 급등하고 있...,0.75,https://n.news.naver.com/mnews/article/008/000...


In [29]:
candi_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   news_id       50 non-null     object 
 1   wdate         50 non-null     object 
 2   title         50 non-null     object 
 3   image         50 non-null     object 
 4   press         50 non-null     object 
 5   summary       50 non-null     object 
 6   impact_score  50 non-null     float64
 7   url           50 non-null     object 
dtypes: float64(1), object(7)
memory usage: 3.2+ KB


5. onnx 모델 불러와서 돌려 two-tower -> 클릭 뉴스 임베딩(n), 후보 뉴스 임베딩(n) 리턴은 후보뉴스 여러개의 재정렬된 리스트

In [30]:
import requests
import time
import numpy as np

API_BASE_URL = "http://15.165.211.100:8000"
HIGHLIGHTDATA_ENDPOINT = "/plm/embedding"

# 1. 임베딩 함수 정의 (배치 처리 포함)
def get_embedding(articles):
    """단일 배치에 대한 임베딩 요청"""
    url = API_BASE_URL + HIGHLIGHTDATA_ENDPOINT
    headers = {"Content-Type": "application/json"}
    response = requests.post(url, json={"articles": articles})
    response.raise_for_status()
    return response.json()

def get_embedding_batch(article_list, batch_size=20):
    """전체 리스트를 배치로 처리"""
    all_embeddings = []
    for i in range(0, len(article_list), batch_size):
        batch = article_list[i:i+batch_size]
        try:
            result = get_embedding(batch)
            batch_embeddings = result.get("embeddings", [])
            all_embeddings.extend(batch_embeddings)
            print(f"배치 {i}-{i+len(batch)}: {len(batch_embeddings)}개 임베딩 성공")
        except Exception as e:
            print(f"배치 {i}-{i+len(batch)} 실패: {str(e)}")
            all_embeddings.extend([None] * len(batch))
        time.sleep(0.3)  # 서버 부하 방지
    return all_embeddings

# 2. 데이터 추출
clicked_summary_list = clicked_df['summary'].tolist()
candi_summary_list = candi_df['summary'].tolist()

# 3. 배치 처리로 임베딩 수행
clicked_embeddings = get_embedding_batch(clicked_summary_list)
candi_embeddings = get_embedding_batch(candi_summary_list)

# 4. 임베딩 결과를 DataFrame에 추가
clicked_df["embedding"] = clicked_embeddings
candi_df["embedding"] = candi_embeddings

# 5. 결과 확인
print(f"Clicked_df 임베딩 길이: {len(clicked_embeddings)}")
print(f"Candi_df 임베딩 길이: {len(candi_embeddings)}")
print(clicked_df[['news_id', 'embedding']].head())


배치 0-20: 20개 임베딩 성공
배치 20-30: 10개 임베딩 성공
배치 0-20: 20개 임베딩 성공
배치 20-40: 20개 임베딩 성공
배치 40-50: 10개 임베딩 성공
Clicked_df 임베딩 길이: 30
Candi_df 임베딩 길이: 50
         news_id                                          embedding
0  20250523_0028  [-0.6349802017211914, 0.04213537648320198, -0....
1  20250523_0067  [-0.23859785497188568, -0.4166261851787567, -0...
2  20250523_0075  [-0.35290783643722534, -0.693975031375885, -0....
3  20250523_0087  [0.08004584163427353, -0.5533429384231567, -0....
4  20250523_0109  [-0.6267953515052795, 0.1835927516222, -0.3006...


In [55]:
import onnxruntime as ort
import numpy as np
import pandas as pd

# 1. ONNX 모델 로드 함수
def load_onnx_model(model_path: str):
    try:
        session_options = ort.SessionOptions()
        session_options.graph_optimization_level = ort.GraphOptimizationLevel.ORT_ENABLE_ALL
        session = ort.InferenceSession(model_path, session_options)
        print("✅ ONNX 모델 로드 성공")
        return session
    except Exception as e:
        print(f"❌ 모델 로드 실패: {str(e)}")
        raise

# 2. 추론 함수
def predict_scores(session, clicked_embeddings: np.ndarray, candidate_embeddings: np.ndarray):
    inputs = {
        "clicked": clicked_embeddings.astype(np.float32),
        "candidates": candidate_embeddings.astype(np.float32)
    }
    try:
        outputs = session.run(None, inputs)
        return outputs[0]  # [batch_size, num_candidates]
    except Exception as e:
        print(f"❌ 추론 오류: {str(e)}")
        raise

# 3. 실제 데이터 처리 및 추론
def run_inference(clicked_df: pd.DataFrame, candi_df: pd.DataFrame, model_path: str, num_clicked=3, num_candidates=5, top_k=3):
    # 모델 로드
    session = load_onnx_model(model_path)
    
    # 임베딩 데이터 추출
    clicked_embeddings_np = np.array(clicked_df["embedding"].tolist(), dtype=np.float32)
    candi_embeddings_np = np.array(candi_df["embedding"].tolist(), dtype=np.float32)
    
    # 입력 데이터 준비 (슬라이싱 및 차원 조정)
    clicked_input = clicked_embeddings_np[:num_clicked].reshape(1, num_clicked, 768)
    candidate_input = candi_embeddings_np[:num_candidates].reshape(1, num_candidates, 768)
    
    # 추론 실행
    scores = predict_scores(session, clicked_input, candidate_input)
    
    # 결과 처리
    print("추천 점수 형태:", scores.shape)
    print("추천 점수:", scores)
    
    # Top-K 후보 선정
    top_indices = np.argsort(-scores, axis=2)[:, :, :top_k]

    print(f"Top-{top_k} 후보 인덱스:", top_indices)

    # 1) 배치 차원 제거하고 펼치기
    flattened = top_indices.squeeze(0).flatten()
    print(f"전체 후보군 (중복 포함): {flattened}")

    # 2) 유니크 후보군 추출
    unique_candidates = np.unique(flattened)
    print(f"중복 제거 후보군: {unique_candidates}")

    # 3) 등장 빈도 세기
    counts = Counter(flattened)
    print(f"등장 횟수: {counts}")

    # 후보군 리스트와 각 후보의 가중치 리스트 만들기
    candidate_list = []
    weight_list = []

    for candidate in unique_candidates:
        weight = counts[candidate]  # 등장 횟수를 가중치로!
        candidate_list.append(candidate)
        weight_list.append(weight)

    print("최종 후보 리스트:", candidate_list)
    print("각 후보 가중치:", weight_list)

    # ✅ 여기서 뉴스ID 리스트 + 등장 횟수 dict 만들기!
    candidate_news_ids = candidate_list
    appear_counts = weight_list

    appear_count_map = dict(zip(candidate_news_ids, appear_counts))

    print("✅ 후보군 뉴스ID 리스트:", candidate_news_ids)
    print("✅ 등장 횟수 리스트:", appear_counts)
    print("✅ 등장 횟수 dict:", appear_count_map)

    return scores, top_indices, candidate_news_ids, appear_counts, appear_count_map




In [57]:
from collections import Counter

MODEL_PATH = "C:/Users/user/fin_project/modelapi/models/two_tower_model.onnx"
    
# 실제 실행
scores, top_indices, candidate_news_ids, appear_counts, appear_count_map = run_inference(
    clicked_df,
    candi_df,
    model_path=MODEL_PATH,
    num_clicked=len(clicked_df),
    num_candidates=len(candi_df),
    top_k=3
)



✅ ONNX 모델 로드 성공
추천 점수 형태: (1, 30, 50)
추천 점수: [[[-1.0541983   0.46712148  1.6332407  ... -0.47988525  1.9477448
    0.52488065]
  [-0.7966115   0.41932654  1.1363248  ... -0.3978112   1.387647
    0.4442144 ]
  [-0.77376634  0.33859178  1.3478248  ... -0.32643723  1.6580838
    0.42728412]
  ...
  [-0.5276992  -0.9889669   0.10808378 ... -0.03807717 -0.3036973
   -1.06347   ]
  [ 0.0702047   0.30071238 -0.29350832 ...  0.17718838  0.41003305
    0.65169626]
  [ 0.10197181 -0.74727094 -1.0132447  ...  0.37194297 -0.7877817
   -0.39453614]]]
Top-3 후보 인덱스: [[[10  9 24]
  [10  9 24]
  [10 36  9]
  [26 44 42]
  [10  9 24]
  [ 3  7 10]
  [25  8 35]
  [10 24  9]
  [ 7  3 47]
  [ 1 19 20]
  [18 25  8]
  [10 36 48]
  [25 35  8]
  [10  9 48]
  [10  9 24]
  [10  9 24]
  [10  9 24]
  [10  9 24]
  [ 3  7 10]
  [25  8 35]
  [25 15 35]
  [26 18 39]
  [42 44 26]
  [25 35  8]
  [26 27 39]
  [35 19 28]
  [ 8 35 25]
  [ 7 45  3]
  [15 25 16]
  [26 27 39]]]
전체 후보군 (중복 포함): [10  9 24 10  9 24 10 36  9 26 44

In [61]:
final_news_ids = candi_df.iloc[candidate_news_ids]['news_id'].tolist()
print("✅ 실제 뉴스ID 리스트:", final_news_ids)

# 🔹 그리고 등장 횟수 리스트는 이미 appear_counts 로 있음!
print("✅ 가중치 리스트:", appear_counts)

appear_count_map = dict(zip(final_news_ids, appear_counts))
print("👉 뉴스ID ➝ 등장 횟수 dict:", appear_count_map)


✅ 실제 뉴스ID 리스트: ['20250414_0083', '20250513_0148', '20250513_0041', '20250417_0067', '20250430_0098', '20250430_0083', '20250331_0086', '20250331_0106', '20250425_0045', '20250512_0093', '20250326_0027', '20250307_0027', '20250421_0033', '20250508_0042', '20250508_0043', '20250401_0070', '20250407_0049', '20250422_0105', '20250509_0178', '20250219_0116', '20250219_0175', '20250424_0028', '20250219_0162', '20250409_0137']
✅ 가중치 리스트: [1, 4, 4, 6, 10, 13, 2, 1, 2, 2, 1, 8, 8, 5, 2, 1, 7, 2, 3, 2, 2, 1, 1, 2]
👉 뉴스ID ➝ 등장 횟수 dict: {'20250414_0083': 1, '20250513_0148': 4, '20250513_0041': 4, '20250417_0067': 6, '20250430_0098': 10, '20250430_0083': 13, '20250331_0086': 2, '20250331_0106': 1, '20250425_0045': 2, '20250512_0093': 2, '20250326_0027': 1, '20250307_0027': 8, '20250421_0033': 8, '20250508_0042': 5, '20250508_0043': 2, '20250401_0070': 1, '20250407_0049': 7, '20250422_0105': 2, '20250509_0178': 3, '20250219_0116': 2, '20250219_0175': 2, '20250424_0028': 1, '20250219_0162': 1, '20250

# Step2

-----------------------------------------------

1. 유저 아이디 이용해서 유저 정보 조회 api로 userPnl, asset, investScore 가져오기

In [94]:
import pandas as pd


############ 이부분 api 고쳐지면 이 코드로 바꾸기 ################
# def fetch_userinfo_to_df_and_merge(member_id: str, click_log_df: pd.DataFrame) -> pd.DataFrame:
#     """
#     API에서 유저 정보를 받아와서 DataFrame으로 만들고,
#     click_log_df와 user_id 기준으로 병합
#     """
#     # API 호출 및 DataFrame 변환
#     url = f"http://43.200.17.139:8080/api/v1/userinfo/{member_id}"
#     response = requests.get(url)
#     response.raise_for_status()
#     userinfo = response.json()
#     user_df = pd.DataFrame([userinfo])

#     # user_id 컬럼이 없으면 추가 (memberId를 user_id로 맞춰줌)
#     if 'user_id' not in user_df.columns and 'memberId' in user_df.columns:
#         user_df = user_df.rename(columns={'memberId': 'user_id'})
#     if 'user_id' not in click_log_df.columns and 'memberId' in click_log_df.columns:
#         click_log_df = click_log_df.rename(columns={'memberId': 'user_id'})

#     # 병합
#     merged_df = pd.merge(click_log_df, user_df, on='user_id', how='left')

#     return merged_df

# # click_log_df는 이미 선언되어 있다고 가정
# member_id = "u1"
# result_df = fetch_userinfo_to_df_and_merge(member_id, click_log_df)
# print(result_df.head())


###############################################################
# api 고장나있어서 하드코딩한 부분

import pandas as pd

def fetch_userinfo_to_df_and_merge_sample(click_log_df: pd.DataFrame, sample_userinfo: dict) -> pd.DataFrame:
    """
    sample_userinfo 딕셔너리에서 유저 정보 DataFrame 만들고,
    click_log_df와 user_id 기준으로 병합
    """
    # sample_userinfo를 DataFrame으로 변환
    user_df = pd.DataFrame([sample_userinfo])
    
    # user_id 컬럼이 없으면 memberId를 user_id로 바꾸기
    if 'user_id' not in user_df.columns and 'memberId' in user_df.columns:
        user_df = user_df.rename(columns={'memberId': 'user_id'})
    
    if 'user_id' not in click_log_df.columns and 'memberId' in click_log_df.columns:
        click_log_df = click_log_df.rename(columns={'memberId': 'user_id'})
    
    # 병합 (user_id 기준, 왼쪽 click_log_df 유지)
    merged_df = pd.merge(click_log_df, user_df, on='user_id', how='left')
    
    return merged_df


sample_userinfo = {
  "memberId": "u1",
  "username": "test",
  "userPnl": -220000,
  "asset": 7000000,
  "investScore": 4,
  "memberStocks": [
    {
      "stockCode": "019170",
      "stockName": "신풍제약",
      "pnl": 100000
    },
    {
      "stockCode": "000100",
      "stockName": "유한양행",
      "pnl": 100000
    }
  ]
}
merged_df = fetch_userinfo_to_df_and_merge_sample(click_log_df, sample_userinfo)
merged_df


,Unnamed: 0,user_id,news_id,clicked,wdate,stock_list,industry_list,impact_score,topic,username,userPnl,asset,investScore,memberStocks
0,0,u1,20250523_0002,0,2025-05-23 18:52:00,"[{'stock_id': '377300', 'stock_name': '카카오페이'}]","[{'stock_id': '377300', 'industry_id': '116601...",0.33,4,test,-220000,7000000,4,"[{'stockCode': '019170', 'stockName': '신풍제약', ..."
1,1,u1,20250523_0004,0,2025-05-23 18:33:00,"[{'stock_id': '005930', 'stock_name': '삼성전자'}]","[{'stock_id': '005930', 'industry_id': '32604'...",0.05,9,test,-220000,7000000,4,"[{'stockCode': '019170', 'stockName': '신풍제약', ..."
2,2,u1,20250523_0007,0,2025-05-23 18:00:00,"[{'stock_id': '377300', 'stock_name': '카카오페이'}]","[{'stock_id': '377300', 'industry_id': '116601...",0.33,4,test,-220000,7000000,4,"[{'stockCode': '019170', 'stockName': '신풍제약', ..."
3,3,u1,20250523_0010,0,2025-05-23 17:52:00,"[{'stock_id': '298040', 'stock_name': '효성중공업'}]","[{'stock_id': '298040', 'industry_id': '32801'...",0.08,9,test,-220000,7000000,4,"[{'stockCode': '019170', 'stockName': '신풍제약', ..."
4,4,u1,20250523_0011,0,2025-05-23 17:52:00,"[{'stock_id': '377300', 'stock_name': '카카오페이'}]","[{'stock_id': '377300', 'industry_id': '116601...",0.33,4,test,-220000,7000000,4,"[{'stockCode': '019170', 'stockName': '신풍제약', ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13377,13377,u1,20240112_0215,0,2024-01-12 08:14:00,"[{'stock_id': '005930', 'stock_name': '삼성전자'}]","[{'stock_id': '005930', 'industry_id': '32604'...",0.08,3,test,-220000,7000000,4,"[{'stockCode': '019170', 'stockName': '신풍제약', ..."
13378,13378,u1,20240112_0221,0,2024-01-12 08:11:00,"[{'stock_id': '039490', 'stock_name': '키움증권'}]","[{'stock_id': '039490', 'industry_id': '116601...",0.03,2,test,-220000,7000000,4,"[{'stockCode': '019170', 'stockName': '신풍제약', ..."
13379,13379,u1,20240112_0223,0,2024-01-12 08:09:00,"[{'stock_id': '375500', 'stock_name': 'DL이앤씨'}]","[{'stock_id': '375500', 'industry_id': '64102'...",0.09,3,test,-220000,7000000,4,"[{'stockCode': '019170', 'stockName': '신풍제약', ..."
13380,13380,u1,20240112_0224,0,2024-01-12 08:09:00,"[{'stock_id': '005940', 'stock_name': 'NH투자증권'}]","[{'stock_id': '005940', 'industry_id': '116601...",0.04,2,test,-220000,7000000,4,"[{'stockCode': '019170', 'stockName': '신풍제약', ..."


2. 후보군 뉴스 상세 메타데이터 조회로 스톡리스트, 인더스트리 리스트, 임팩트 스코어, 토픽 db에서 끌고와서 붙이고 가장 큰값으로 메인토픽도 붙이기

In [111]:
import asyncio
import httpx
import pandas as pd
import nest_asyncio  # Jupyter 환경 호환용

# API 기본 설정
API_BASE_URL = "http://3.37.207.16:8000"
METADATA_ENDPOINT = "/news/v2/{news_id}/metadata"

async def fetch_news_metadata(news_id: str) -> dict:
    """
    개별 뉴스 ID로 메타데이터 조회 (비동기)
    - 성공 시: API 응답 반환
    - 실패 시: 에러 정보 포함한 딕셔너리 반환
    """
    url = f"{API_BASE_URL}{METADATA_ENDPOINT.format(news_id=news_id)}"
    async with httpx.AsyncClient(timeout=10.0) as client:
        try:
            response = await client.get(url)
            response.raise_for_status()  # 4xx/5xx 상태 코드 예외 발생
            return response.json() | {"news_id": news_id}  # news_id 병합
        except httpx.HTTPStatusError as e:
            return {
                "news_id": news_id,
                "error": f"HTTP {e.response.status_code}",
                "detail": str(e)
            }
        except Exception as e:
            return {
                "news_id": news_id,
                "error": "ConnectionError",
                "detail": str(e)
            }

async def fetch_all_metadata(news_ids: list) -> list:
    """병렬 처리로 모든 뉴스 메타데이터 수집"""
    tasks = [fetch_news_metadata(nid) for nid in news_ids]
    return await asyncio.gather(*tasks)

def create_metadata_df(metadata_list: list) -> pd.DataFrame:
    """수집된 메타데이터를 데이터프레임으로 변환"""
    # 필드별 데이터 추출
    processed = []
    for item in metadata_list:
        # 에러 발생 시 처리
        if "error" in item:
            processed.append({
                "news_id": item["news_id"],
                "error": item["error"],
                "summary": None,
                # "stock_list": None,
                "stock_list_view": None
                # "industry_list": None
            })
            continue
        
        # 정상 응답 처리
        processed.append({
            "news_id": item["news_id"],
            "summary": item.get("summary", ""),
            # "stock_list": item.get("stock_list", []),
            "stock_list_view": item.get("stock_list_view", [])
            # "industry_list": item.get("industry_list", [])
        })
    
    return pd.DataFrame(processed)

# Jupyter 환경 호환 설정
nest_asyncio.apply()

# 예시 실행 (news_id_list는 이전 단계에서 추출된 리스트)
# news_id_list = ['20250513_0092', '20250618_28735495']
metadata_list = asyncio.get_event_loop().run_until_complete(
    fetch_all_metadata(final_news_ids)
)
metadata_df = create_metadata_df(metadata_list)

# 결과 확인
metadata_df


,news_id,summary,stock_list_view
0,20250414_0083,윤석열 전 대통령 탄핵 이후 치러지는 조기 대선 유력 후보로 꼽혀 오세훈 서울시장이...,"[{'stock_id': '051630', 'stock_name': '진양화학'},..."
1,20250513_0148,호반건설이 12일 한진칼 보유 지분을 17.44%에서 18.46%로 1.02%포인트...,"[{'stock_id': '003490', 'stock_name': '대한항공'},..."
2,20250513_0041,호반건설이 한진칼 지분을 추가로 매입해 지분율(특수관계인 포함)이 17.44%에서 ...,"[{'stock_id': '180640', 'stock_name': '한진칼'}, ..."
3,20250417_0067,제이준코스메틱은 연어나 송어의 생식세포에서 DNA 조각을 추출한 물질인 PDRN(폴...,"[{'stock_id': '025620', 'stock_name': '제이준코스메틱'}]"
4,20250430_0098,웅진이 국내 1위 상조 업체인 프리드라이프를 인수한다는 소식에 장 초반 급등하고 있...,"[{'stock_id': '016880', 'stock_name': '웅진'}]"
5,20250430_0083,웅진이 종속회사 WJ라이프가 상조회사 프리드라이프 지분 99.77%를 인수하는 주식...,"[{'stock_id': '016880', 'stock_name': '웅진'}, {..."
6,20250331_0086,한솔홀딩스의 한솔PNS 상장폐지를 위한 공개매수 소식이 알려지자 주가가 상승세를 보...,"[{'stock_id': '010420', 'stock_name': '한솔PNS'}..."
7,20250331_0106,"한솔홀딩스는 한솔PNS의 지분 46.07%를 보유한 1대 주주로, 이번 공개매수로 ...","[{'stock_id': '010420', 'stock_name': '한솔PNS'}..."
8,20250425_0045,지난 24일 글로벌 2차전지 기업 CATL이 2세대 나트륨 이온 배터리를 공개하며 ...,"[{'stock_id': '161000', 'stock_name': '애경케미칼'}]"
9,20250512_0093,국민의힘 대선 후보로 김문수 후보가 최종 확정되자 평화홀딩스와 평화산업은 계열사가 ...,"[{'stock_id': '010770', 'stock_name': '평화홀딩스'}..."


In [112]:
metadata_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   news_id          24 non-null     object
 1   summary          24 non-null     object
 2   stock_list_view  24 non-null     object
dtypes: object(3)
memory usage: 704.0+ bytes


In [129]:
# news_id 기준으로 metadata_df와 merged_df 병합 (inner join)
final_df = pd.merge(metadata_df, merged_df, on='news_id', how="inner")

In [130]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   news_id          24 non-null     object 
 1   summary          24 non-null     object 
 2   stock_list_view  24 non-null     object 
 3   Unnamed: 0       24 non-null     int64  
 4   user_id          24 non-null     object 
 5   clicked          24 non-null     int64  
 6   wdate            24 non-null     object 
 7   stock_list       24 non-null     object 
 8   industry_list    24 non-null     object 
 9   impact_score     24 non-null     float64
 10  topic            24 non-null     int64  
 11  username         24 non-null     object 
 12  userPnl          24 non-null     int64  
 13  asset            24 non-null     int64  
 14  investScore      24 non-null     int64  
 15  memberStocks     24 non-null     object 
dtypes: float64(1), int64(6), object(9)
memory usage: 3.1+ KB


In [131]:
final_df

,news_id,summary,stock_list_view,Unnamed: 0,user_id,clicked,wdate,stock_list,industry_list,impact_score,topic,username,userPnl,asset,investScore,memberStocks
0,20250414_0083,윤석열 전 대통령 탄핵 이후 치러지는 조기 대선 유력 후보로 꼽혀 오세훈 서울시장이...,"[{'stock_id': '051630', 'stock_name': '진양화학'},...",1064,u1,0,2025-04-14 09:42:00,"[{'stock_id': '051630', 'stock_name': '진양화학'}]","[{'stock_id': '051630', 'industry_id': '32202'...",0.86,5,test,-220000,7000000,4,"[{'stockCode': '019170', 'stockName': '신풍제약', ..."
1,20250513_0148,호반건설이 12일 한진칼 보유 지분을 17.44%에서 18.46%로 1.02%포인트...,"[{'stock_id': '003490', 'stock_name': '대한항공'},...",362,u1,0,2025-05-13 00:25:00,"[{'stock_id': '180640', 'stock_name': '한진칼'}]","[{'stock_id': '180640', 'industry_id': '116409...",0.78,9,test,-220000,7000000,4,"[{'stockCode': '019170', 'stockName': '신풍제약', ..."
2,20250513_0041,호반건설이 한진칼 지분을 추가로 매입해 지분율(특수관계인 포함)이 17.44%에서 ...,"[{'stock_id': '180640', 'stock_name': '한진칼'}, ...",333,u1,0,2025-05-13 15:00:00,"[{'stock_id': '180640', 'stock_name': '한진칼'}]","[{'stock_id': '180640', 'industry_id': '116409...",0.78,9,test,-220000,7000000,4,"[{'stockCode': '019170', 'stockName': '신풍제약', ..."
3,20250417_0067,제이준코스메틱은 연어나 송어의 생식세포에서 DNA 조각을 추출한 물질인 PDRN(폴...,"[{'stock_id': '025620', 'stock_name': '제이준코스메틱'}]",938,u1,1,2025-04-17 10:01:00,"[{'stock_id': '025620', 'stock_name': '제이준코스메틱'}]","[{'stock_id': '025620', 'industry_id': '32004'...",0.75,1,test,-220000,7000000,4,"[{'stockCode': '019170', 'stockName': '신풍제약', ..."
4,20250430_0098,웅진이 국내 1위 상조 업체인 프리드라이프를 인수한다는 소식에 장 초반 급등하고 있...,"[{'stock_id': '016880', 'stock_name': '웅진'}]",581,u1,1,2025-04-30 09:38:00,"[{'stock_id': '016880', 'stock_name': '웅진'}]","[{'stock_id': '016880', 'industry_id': '137105...",0.75,6,test,-220000,7000000,4,"[{'stockCode': '019170', 'stockName': '신풍제약', ..."
5,20250430_0083,웅진이 종속회사 WJ라이프가 상조회사 프리드라이프 지분 99.77%를 인수하는 주식...,"[{'stock_id': '016880', 'stock_name': '웅진'}, {...",580,u1,0,2025-04-30 10:08:00,"[{'stock_id': '016880', 'stock_name': '웅진'}]","[{'stock_id': '016880', 'industry_id': '137105...",0.75,9,test,-220000,7000000,4,"[{'stockCode': '019170', 'stockName': '신풍제약', ..."
6,20250331_0086,한솔홀딩스의 한솔PNS 상장폐지를 위한 공개매수 소식이 알려지자 주가가 상승세를 보...,"[{'stock_id': '010420', 'stock_name': '한솔PNS'}...",1410,u1,0,2025-03-31 10:28:00,"[{'stock_id': '004150', 'stock_name': '한솔홀딩스'}...","[{'stock_id': '004150', 'industry_id': '116409...",0.59,7,test,-220000,7000000,4,"[{'stockCode': '019170', 'stockName': '신풍제약', ..."
7,20250331_0106,"한솔홀딩스는 한솔PNS의 지분 46.07%를 보유한 1대 주주로, 이번 공개매수로 ...","[{'stock_id': '010420', 'stock_name': '한솔PNS'}...",1412,u1,0,2025-03-31 09:39:00,"[{'stock_id': '004150', 'stock_name': '한솔홀딩스'}...","[{'stock_id': '004150', 'industry_id': '116409...",0.59,9,test,-220000,7000000,4,"[{'stockCode': '019170', 'stockName': '신풍제약', ..."
8,20250425_0045,지난 24일 글로벌 2차전지 기업 CATL이 2세대 나트륨 이온 배터리를 공개하며 ...,"[{'stock_id': '161000', 'stock_name': '애경케미칼'}]",700,u1,1,2025-04-25 11:03:00,"[{'stock_id': '161000', 'stock_name': '애경케미칼'}]","[{'stock_id': '161000', 'industry_id': '32001'...",0.57,6,test,-220000,7000000,4,"[{'stockCode': '019170', 'stockName': '신풍제약', ..."
9,20250512_0093,국민의힘 대선 후보로 김문수 후보가 최종 확정되자 평화홀딩스와 평화산업은 계열사가 ...,"[{'stock_id': '010770', 'stock_name': '평화홀딩스'}...",381,u1,0,2025-05-12 09:35:00,"[{'stock_id': '010770', 'stock_name': '평화홀딩스'}...","[{'stock_id': '010770', 'industry_id': '33003'...",0.57,5,test,-220000,7000000,4,"[{'stockCode': '019170', 'stockName': '신풍제약', ..."


In [132]:
import pandas as pd
import psycopg2


def get_topic_data(news_ids: list) -> pd.DataFrame:
    """
    psycopg2로 직접 DB 연결 후,
    news_id 목록으로 토픽 데이터 조회해서 DataFrame 반환
    """
    if not news_ids:
        return pd.DataFrame()

    # DB 연결
    conn = psycopg2.connect(
        host='3.37.207.16',
        port='5432',
        dbname='postgres',
        user='postgres',
        password='password'
    )
    cursor = conn.cursor()

    try:
        # 안전한 방식으로 IN 절에 리스트 전달
        query = """
        SELECT news_id, topic_1, topic_2, topic_3, topic_4, topic_5,
               topic_6, topic_7, topic_8, topic_9
        FROM news_v2_topic
        WHERE news_id IN %s
        """
        
        # 리스트를 튜플로 변환하여 매개변수로 전달
        cursor.execute(query, (tuple(news_ids),))
        rows = cursor.fetchall()
        
        # 컬럼명 정의
        columns = ['news_id', 'topic_1', 'topic_2', 'topic_3', 'topic_4',
                   'topic_5', 'topic_6', 'topic_7', 'topic_8', 'topic_9']
        
        # DataFrame 생성
        df = pd.DataFrame(rows, columns=columns)
        
    except Exception as e:
        print(f"Database query error: {e}")
        df = pd.DataFrame()
    
    finally:
        cursor.close()
        conn.close()

    return df


def add_main_topic(df: pd.DataFrame) -> pd.DataFrame:
    """
    토픽 점수 중 가장 큰 값으로 메인 토픽 선정 (숫자만 저장)
    """
    topic_columns = [f'topic_{i}' for i in range(1, 10)]
    
    # 토픽 데이터가 있는 행만 처리
    has_topic_data = df[topic_columns].notna().any(axis=1)
    
    if has_topic_data.any():
        # 1. 최대값을 가진 컬럼명 추출 ('topic_5' 형태)
        main_topic_str = df.loc[has_topic_data, topic_columns].idxmax(axis=1)
        
        # 2. 숫자만 추출하여 Series로 변환
        main_topic_series = main_topic_str.str.extract(r'(\d+)', expand=False).astype(int)
        
        # 3. 변환된 Series를 main_topic 컬럼에 할당
        df.loc[has_topic_data, 'main_topic'] = main_topic_series
    
    return df



In [133]:
# 1. final_df에서 news_id 리스트 추출
news_ids = final_df['news_id'].tolist()

# 2. 토픽 데이터 조회
topic_df = get_topic_data(news_ids)

# 3. 메인 토픽 컬럼 추가
topic_df = add_main_topic(topic_df)

# 4. metadata_df와 topic_df를 news_id 기준으로 병합
result_df = final_df.merge(topic_df, on='news_id', how='left')

# is_same_stock, is_same_industry 계산 함수
def check_overlap(news_list, user_list, key):
    news_ids = {item[key] for item in news_list}
    user_ids = {item[key] for item in user_list}
    return int(len(news_ids & user_ids) > 0)

# 적용
def check_overlap_manual(stock_list, memberStocks):
    # news side stock_id 리스트
    news_stock_ids = {stock['stock_id'] for stock in stock_list}
    # user side stockCode 리스트
    user_stock_codes = {stock['stockCode'] for stock in memberStocks}

    # 교집합 있으면 1, 없으면 0 리턴
    return 1 if news_stock_ids & user_stock_codes else 0

result_df["is_same_stock"] = result_df.apply(
    lambda row: check_overlap_manual(row["stock_list_view"], row["memberStocks"]),
    axis=1,
)



In [134]:
result_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 27 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   news_id          24 non-null     object 
 1   summary          24 non-null     object 
 2   stock_list_view  24 non-null     object 
 3   Unnamed: 0       24 non-null     int64  
 4   user_id          24 non-null     object 
 5   clicked          24 non-null     int64  
 6   wdate            24 non-null     object 
 7   stock_list       24 non-null     object 
 8   industry_list    24 non-null     object 
 9   impact_score     24 non-null     float64
 10  topic            24 non-null     int64  
 11  username         24 non-null     object 
 12  userPnl          24 non-null     int64  
 13  asset            24 non-null     int64  
 14  investScore      24 non-null     int64  
 15  memberStocks     24 non-null     object 
 16  topic_1          24 non-null     float64
 17  topic_2          2

In [135]:
columns_to_keep = [
    'userPnl', 'asset', 'investScore', 'topic_1', 'topic_2', 'topic_3', 'topic_4',
    'topic_5', 'topic_6', 'topic_7', 'topic_8', 'topic_9', 'main_topic',
    'is_same_stock', 'is_same_industry', 'clicked'
]

pred_df = result_df[[col for col in columns_to_keep if col in result_df.columns]].copy()


In [136]:
pred_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   userPnl        24 non-null     int64  
 1   asset          24 non-null     int64  
 2   investScore    24 non-null     int64  
 3   topic_1        24 non-null     float64
 4   topic_2        24 non-null     float64
 5   topic_3        24 non-null     float64
 6   topic_4        24 non-null     float64
 7   topic_5        24 non-null     float64
 8   topic_6        24 non-null     float64
 9   topic_7        24 non-null     float64
 10  topic_8        24 non-null     float64
 11  topic_9        24 non-null     float64
 12  main_topic     24 non-null     float64
 13  is_same_stock  24 non-null     int64  
 14  clicked        24 non-null     int64  
dtypes: float64(10), int64(5)
memory usage: 2.9 KB


4. 나온 결과로 내 모델에 넣어서 리랭킹 4개

In [ ]:
def get_top_recommendations(result_df, top_k=4):
    """
    LightGBM 모델로 상위 K개 뉴스 추천
    """
    model = load_lgbm_model()
    if model is None:
        print("❌ 모델 로드 실패")
        return None
    
    features_df = result_df.copy()
    
    print(f"피처 데이터 형태: {features_df.shape}")
    print(f"컬럼: {list(features_df.columns)}")
    
    try:
        proba = model.predict_proba(features_df.drop(columns=['clicked']))[:, 1]
        
        result_df = result_df.copy()
        result_df['click_probability'] = proba
        
        # result_df = result_df.sort_values('click_probability', ascending=False)
        
        # top_recommendations = result_df.head(top_k)
        
        # print(f"✅ 상위 {top_k}개 추천 완료")
        # for idx, row in top_recommendations.iterrows():
        #     print(f"  인덱스: {idx}, 확률: {row['click_probability']:.4f}")
        
        return top_recommendations
    
    except Exception as e:
        print(f"❌ 예측 중 오류: {e}")
        return None


def test_lgbm_recommendation(result_df):
    """
    LightGBM 추천 테스트 (result_df 그대로 받아서 씀)
    """
    recommendations = get_top_recommendations(result_df, top_k=4)
    
    if recommendations is not None:
        print("\n=== 최종 추천 결과 ===")
        print(recommendations[['click_probability', 'is_same_stock', 'main_topic']].head())
        return recommendations
    else:
        print("추천 실패")
        return None


In [138]:
# 추천 함수 실행
top_k = len(pred_df)
recommendations = get_top_recommendations(pred_df, top_k=top_k)

if recommendations is not None:
    print("\n=== 최종 추천 결과 ===")
    print(recommendations['clicked'])
else:
    print("추천 실패")

모델 로드 성공: ../../modelapi/models/lgbm_model2.pkl
피처 데이터 형태: (24, 15)
컬럼: ['userPnl', 'asset', 'investScore', 'topic_1', 'topic_2', 'topic_3', 'topic_4', 'topic_5', 'topic_6', 'topic_7', 'topic_8', 'topic_9', 'main_topic', 'is_same_stock', 'clicked']

=== 최종 추천 결과 ===
0     0
1     0
2     0
3     1
4     1
5     0
6     0
7     0
8     1
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    1
18    1
19    0
20    0
21    1
22    1
23    0
Name: clicked, dtype: int64


In [139]:
recommendations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   userPnl            24 non-null     int64  
 1   asset              24 non-null     int64  
 2   investScore        24 non-null     int64  
 3   topic_1            24 non-null     float64
 4   topic_2            24 non-null     float64
 5   topic_3            24 non-null     float64
 6   topic_4            24 non-null     float64
 7   topic_5            24 non-null     float64
 8   topic_6            24 non-null     float64
 9   topic_7            24 non-null     float64
 10  topic_8            24 non-null     float64
 11  topic_9            24 non-null     float64
 12  main_topic         24 non-null     float64
 13  is_same_stock      24 non-null     int64  
 14  clicked            24 non-null     int64  
 15  click_probability  24 non-null     float64
dtypes: float64(11), int64(5)
mem

In [128]:
recommendations

,userPnl,asset,investScore,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,main_topic,is_same_stock,clicked,click_probability
3,-220000,7000000,4,0.5783,0.0531,0.0526,0.0526,0.0526,0.0526,0.0526,0.0528,0.0526,1.0,0,1,0.865950
18,-220000,7000000,4,0.4133,0.2422,0.0130,0.0127,0.0127,0.0129,0.2678,0.0127,0.0127,1.0,0,1,0.845041
17,-220000,7000000,4,0.0562,0.1373,0.0084,0.0087,0.0086,0.3696,0.1565,0.0085,0.2461,6.0,0,1,0.750659
21,-220000,7000000,4,0.3618,0.0666,0.3304,0.0077,0.0922,0.1186,0.0077,0.0076,0.0075,1.0,0,1,0.717849
22,-220000,7000000,4,0.4845,0.0061,0.3152,0.1641,0.0060,0.0061,0.0060,0.0060,0.0060,1.0,0,1,0.717574
8,-220000,7000000,4,0.2247,0.0145,0.0966,0.0150,0.0146,0.2628,0.2233,0.0146,0.1339,6.0,0,1,0.394262
4,-220000,7000000,4,0.1401,0.0101,0.0102,0.0103,0.0103,0.3164,0.1496,0.1292,0.2239,6.0,0,1,0.309647
5,-220000,7000000,4,0.3403,0.0169,0.0170,0.0170,0.0171,0.0172,0.0170,0.0172,0.5402,9.0,0,0,0.131037
2,-220000,7000000,4,0.0113,0.0627,0.0116,0.0889,0.0115,0.0113,0.0115,0.0113,0.7798,9.0,0,0,0.127687
7,-220000,7000000,4,0.0190,0.0185,0.0186,0.0186,0.0188,0.0188,0.0185,0.0187,0.8504,9.0,0,0,0.121434


In [141]:
real_final_df = pd.concat(
    [result_df,  # 혹시 인덱스 안 맞으면 drop 해주기
     recommendations[['click_probability']]],
    axis=1
)

real_final_df = real_final_df.sort_values('click_probability', ascending=False).reset_index(drop=True)

real_final_df

,news_id,summary,stock_list_view,Unnamed: 0,user_id,clicked,wdate,stock_list,industry_list,impact_score,...,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,main_topic,is_same_stock,click_probability
0,20250417_0067,제이준코스메틱은 연어나 송어의 생식세포에서 DNA 조각을 추출한 물질인 PDRN(폴...,"[{'stock_id': '025620', 'stock_name': '제이준코스메틱'}]",938,u1,1,2025-04-17 10:01:00,"[{'stock_id': '025620', 'stock_name': '제이준코스메틱'}]","[{'stock_id': '025620', 'industry_id': '32004'...",0.75,...,0.0526,0.0526,0.0526,0.0526,0.0526,0.0528,0.0526,1.0,0,0.865950
1,20250509_0178,원본보기 전일 K뷰티주들은 줄줄이 52주 신고가를 경신했으며 에이피알은 양호한 수출...,"[{'stock_id': '278470', 'stock_name': '에이피알'},...",433,u1,1,2025-05-09 06:50:00,"[{'stock_id': '278470', 'stock_name': '에이피알'}]","[{'stock_id': '278470', 'industry_id': '32004'...",0.37,...,0.0130,0.0127,0.0127,0.0129,0.2678,0.0127,0.0127,1.0,0,0.845041
2,20250422_0105,"씨아이테크는 키오스크 사업의 성장 정체를 타개하고, 중장기적으로 기업가치를 제고하기...","[{'stock_id': '004920', 'stock_name': '씨아이테크'}]",830,u1,1,2025-04-22 09:03:00,"[{'stock_id': '004920', 'stock_name': '씨아이테크'}]","[{'stock_id': '004920', 'industry_id': '32605'...",0.40,...,0.0084,0.0087,0.0086,0.3696,0.1565,0.0085,0.2461,6.0,0,0.750659
3,20250424_0028,주요 제품의 공급과잉이 지속되면서 판매가격이 하락했고 대규모 정기보수가 진행된 데 ...,"[{'stock_id': '009830', 'stock_name': '한화솔루션'}]",732,u1,1,2025-04-24 16:15:00,"[{'stock_id': '009830', 'stock_name': '한화솔루션'}]","[{'stock_id': '009830', 'industry_id': '32001'...",0.37,...,0.3304,0.0077,0.0922,0.1186,0.0077,0.0076,0.0075,1.0,0,0.717849
4,20250219_0162,NH투자증권이 지난해 사상 최대 실적을 기록한 풀무원 해외 사업 확대가 기대된다고 ...,"[{'stock_id': '017810', 'stock_name': '풀무원'}, ...",2520,u1,1,2025-02-19 08:23:00,"[{'stock_id': '005940', 'stock_name': 'NH투자증권'...","[{'stock_id': '005940', 'industry_id': '116601...",0.37,...,0.3152,0.1641,0.0060,0.0061,0.0060,0.0060,0.0060,1.0,0,0.717574
5,20250425_0045,지난 24일 글로벌 2차전지 기업 CATL이 2세대 나트륨 이온 배터리를 공개하며 ...,"[{'stock_id': '161000', 'stock_name': '애경케미칼'}]",700,u1,1,2025-04-25 11:03:00,"[{'stock_id': '161000', 'stock_name': '애경케미칼'}]","[{'stock_id': '161000', 'industry_id': '32001'...",0.57,...,0.0966,0.0150,0.0146,0.2628,0.2233,0.0146,0.1339,6.0,0,0.394262
6,20250430_0098,웅진이 국내 1위 상조 업체인 프리드라이프를 인수한다는 소식에 장 초반 급등하고 있...,"[{'stock_id': '016880', 'stock_name': '웅진'}]",581,u1,1,2025-04-30 09:38:00,"[{'stock_id': '016880', 'stock_name': '웅진'}]","[{'stock_id': '016880', 'industry_id': '137105...",0.75,...,0.0102,0.0103,0.0103,0.3164,0.1496,0.1292,0.2239,6.0,0,0.309647
7,20250430_0083,웅진이 종속회사 WJ라이프가 상조회사 프리드라이프 지분 99.77%를 인수하는 주식...,"[{'stock_id': '016880', 'stock_name': '웅진'}, {...",580,u1,0,2025-04-30 10:08:00,"[{'stock_id': '016880', 'stock_name': '웅진'}]","[{'stock_id': '016880', 'industry_id': '137105...",0.75,...,0.0170,0.0170,0.0171,0.0172,0.0170,0.0172,0.5402,9.0,0,0.131037
8,20250513_0041,호반건설이 한진칼 지분을 추가로 매입해 지분율(특수관계인 포함)이 17.44%에서 ...,"[{'stock_id': '180640', 'stock_name': '한진칼'}, ...",333,u1,0,2025-05-13 15:00:00,"[{'stock_id': '180640', 'stock_name': '한진칼'}]","[{'stock_id': '180640', 'industry_id': '116409...",0.78,...,0.0116,0.0889,0.0115,0.0113,0.0115,0.0113,0.7798,9.0,0,0.127687
9,20250331_0106,"한솔홀딩스는 한솔PNS의 지분 46.07%를 보유한 1대 주주로, 이번 공개매수로 ...","[{'stock_id': '010420', 'stock_name': '한솔PNS'}...",1412,u1,0,2025-03-31 09:39:00,"[{'stock_id': '004150', 'stock_name': '한솔홀딩스'}...","[{'stock_id': '004150', 'industry_id': '116409...",0.59,...,0.0186,0.0186,0.0188,0.0188,0.0185,0.0187,0.8504,9.0,0,0.121434
